# Modelado y Validación del Modelo de Regresión

En este notebook se desarrollan los modelos de regresión para predecir la variable objetivo del dataset proporcionado por el curso.

El objetivo es:
- Entrenar distintos modelos de regresión.
- Evaluar su rendimiento mediante métricas estándar (RMSE, MAE, R²).
- Controlar problemas de overfitting y underfitting.
- Seleccionar y guardar el mejor modelo para su uso en la aplicación interactiva.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib


In [2]:
# Carga del dataset procesado
df = pd.read_csv("train_ready_for_modeling.csv")

df.head()



/tmp/ipython-input-2772218163.py:2: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_ready_for_modeling.csv")


,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_Unknown,fuel_type_not supported,fuel_type_–,accident_None reported,clean_title_Yes
0,MINI,Cooper S Base,2007,213000,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,4200.0,False,True,False,False,False,False,False,True,True
1,Lincoln,LS V8,2002,143250,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,4999.0,False,True,False,False,False,False,False,False,True
2,Chevrolet,Silverado 2500 LT,2002,136731,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,13900.0,True,False,False,False,False,False,False,True,True
3,Genesis,G90 5.0 Ultimate,2017,19500,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,45000.0,False,True,False,False,False,False,False,True,True
4,Mercedes-Benz,Metris Base,2021,7388,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,97500.0,False,True,False,False,False,False,False,True,True


#Evaluación de modelos

Para poder comparar de forma consistente los distintos modelos de regresión entrenados, se define una función auxiliar que centraliza el proceso de entrenamiento y evaluación.

Esta función entrena el modelo con el conjunto de entrenamiento y calcula métricas estándar de regresión (RMSE, MAE y R²) tanto en entrenamiento como en test. De esta forma, se puede analizar el rendimiento predictivo de cada modelo y detectar posibles problemas de overfitting o underfitting comparando ambos resultados.

In [3]:
def evaluate_model(model, X_train, y_train, X_test, y_test):

    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Métricas
    metrics = {
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "test_rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "r2": r2_score(y_test, y_test_pred)
    }

    return metrics


## División de datos en Training y Test

El dataset se divide en conjuntos de entrenamiento y prueba con el objetivo de evaluar la capacidad de generalización de los modelos.  
Se utiliza una división 80/20 y una semilla fija para garantizar reproducibilidad.


In [4]:
# Definición de variables predictoras y variable objetivo
X = df.drop("price", axis=1)
y = df["price"]

# Codificación de variables categóricas
X_encoded = pd.get_dummies(X, drop_first=True)  # drop_first=True para evitar multicolinealidad

# División original 80/20 (se usará más adelante para modelos complejos)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(150826, 3603) (37707, 3603) (150826,) (37707,)


## Modelo Base: Regresión Lineal

Se utiliza un modelo de Regresión Lineal como baseline para establecer un punto de referencia.  
Este modelo permite evaluar si modelos más complejos aportan una mejora significativa en el rendimiento.


In [13]:
# Crea copias locales para evitar modificar X_encoded e y globales
X_lr_local = X_encoded.copy()
y_lr_local = y.copy()

# Eliminar las filas donde 'price' (y) sea nulo y las filas correspondientes de X_lr_local
nan_indices_lr = y_lr_local[y_lr_local.isna()].index
X_lr_local = X_lr_local.drop(nan_indices_lr)
y_lr_local = y_lr_local.drop(nan_indices_lr)

# Asegurar de que X_lr_local no tenga nulos. Esto podría ser redundante si get_dummies ya manejó todos los nulos
# o si no había nulos en X_encoded de otras fuentes, pero es una buena medida de seguridad.
X_lr_local = X_lr_local.dropna()
y_lr_local = y_lr_local.loc[X_lr_local.index] # Reajustar y_lr_local después de eliminar de X_lr_local

X_sample, _, y_sample, _ = train_test_split(X_lr_local, y_lr_local, test_size=0.3, random_state=42)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)
# LinearRegression sobre la muestra
lr = LinearRegression()
lr_metrics = evaluate_model(lr, X_train_s, y_train_s, X_test_s, y_test_s)

print("Linear Regression metrics (sampled data):")
print(lr_metrics)


Linear Regression metrics (sampled data):
{'train_rmse': np.float64(70537.49811202243), 'test_rmse': np.float64(91968.62937587018), 'mae': 27331.80993924156, 'r2': 0.027389656552423536}


#Conclusión
El modelo de regresión lineal se utilizó como baseline tras una limpieza explícita de valores faltantes tanto en la variable objetivo como en las variables explicativas.

Los resultados muestran un rendimiento limitado (R² ≈ 0.03) y errores elevados, lo que confirma que la relación entre las características de los vehículos y su precio es altamente no lineal y compleja.

Esta baja capacidad explicativa refuerza la necesidad de emplear modelos más flexibles, como Random Forest o Gradient Boosting, capaces de capturar interacciones y relaciones no lineales presentes en los datos.

## Modelo Ensemble: Random Forest Regressor

Random Forest es un modelo ensemble basado en múltiples árboles de decisión.  
Permite capturar relaciones no lineales y reducir la varianza respecto a modelos individuales.


In [7]:

# Crear copias locales para evitar modificar X_encoded e y globales
X_rf_local = X_encoded.copy()
y_rf_local = y.copy()

# Eliminar las filas donde 'price' (y) sea nulo y las filas correspondientes de X_rf_localnan_indices = y_rf_local[y_rf_local.isna()].index
X_rf_local = X_rf_local.drop(nan_indices)
y_rf_local = y_rf_local.drop(nan_indices)

# Asegurar de que X_rf_local no tenga nulos. Esto podría ser redundante si get_dummies ya manejó todos los nulos
# o si no había nulos en X_encoded de otras fuentes, pero es una buena medida de seguridad.
X_rf_local = X_rf_local.dropna()
y_rf_local = y_rf_local.loc[X_rf_local.index] # Reajustar y_rf_local después de eliminar de X_rf_local

X_rf_sample, _, y_rf_sample, _ = train_test_split(
    X_rf_local, y_rf_local, test_size=0.3, random_state=42
)

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X_rf_sample, y_rf_sample, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=30,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_metrics = evaluate_model(rf, X_train_rf, y_train_rf, X_test_rf, y_test_rf)

print("Random Forest metrics (sampled data):")
rf_metrics


Random Forest metrics (sampled data):


{'train_rmse': np.float64(71356.59135085999),
 'test_rmse': np.float64(88644.26674955047),
 'mae': 23539.44270571143,
 'r2': 0.096432171602572}

## Modelo Ensemble: Gradient Boosting Regressor

Gradient Boosting construye modelos secuenciales que corrigen los errores del modelo anterior.  
Suele ofrecer un buen equilibrio entre sesgo y varianza en problemas de regresión.


In [15]:
# Crear copias locales para evitar modificar X_encoded e y globales
X_gb_local = X_encoded.copy()
y_gb_local = y.copy()

# Eliminar las filas donde 'price' (y) sea nulo y las filas correspondientes de X_gb_local
nan_indices_gb = y_gb_local[y_gb_local.isna()].index
X_gb_local = X_gb_local.drop(nan_indices_gb)
y_gb_local = y_gb_local.drop(nan_indices_gb)

# Asegurar de que X_gb_local no tenga nulos. Esto podría ser redundante si get_dummies ya manejó todos los nulos
# o si no había nulos en X_encoded de otras fuentes, pero es una buena medida de seguridad.
X_gb_local = X_gb_local.dropna()
y_gb_local = y_gb_local.loc[X_gb_local.index] # Reajustar y_gb_local después de eliminar de X_gb_local

# Tomamos una muestra razonable para Gradient Boosting
X_gb_sample, _, y_gb_sample, _ = train_test_split(
    X_gb_local, y_gb_local, test_size=0.3, random_state=42
)
X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(
    X_gb_sample, y_gb_sample, test_size=0.2, random_state=42
)

from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting sobre la muestra
gb = GradientBoostingRegressor(
    n_estimators=50,       # menos árboles para que sea rápido
    max_depth=3,           # profundidad moderada
    learning_rate=0.1,
    random_state=42
)

gb_metrics = evaluate_model(gb, X_train_gb, y_train_gb, X_test_gb, y_test_gb)

print("Gradient Boosting metrics (sampled data):")
print(gb_metrics)


Gradient Boosting metrics (sampled data):
{'train_rmse': np.float64(63574.67337731482), 'test_rmse': np.float64(89137.41449419555), 'mae': 23630.931699233686, 'r2': 0.08635070964365466}


#Modelo Ensemble: XGBoost Regressor
XGBoost es un modelo de gradient boosting optimizado, que construye árboles secuenciales corrigiendo errores del anterior, pero con mejoras de velocidad y regularización.
Suele ofrecer un buen equilibrio entre sesgo y varianza y manejar datasets grandes de manera eficiente

In [17]:
# Importamos el modelo
from xgboost import XGBRegressor

# Crear copias locales para evitar modificar X_encoded e y globales
X_xgb_local = X_encoded.copy()
y_xgb_local = y.copy()

# Eliminar las filas donde 'price' (y) sea nulo y las filas correspondientes de X_xgb_local
nan_indices_xgb = y_xgb_local[y_xgb_local.isna()].index
X_xgb_local = X_xgb_local.drop(nan_indices_xgb)
y_xgb_local = y_xgb_local.drop(nan_indices_xgb)

# Asegurar de que X_xgb_local no tenga nulos. Esto podría ser redundante si get_dummies ya manejó todos los nulos
# o si no había nulos en X_encoded de otras fuentes, pero es una buena medida de seguridad.
X_xgb_local = X_xgb_local.dropna()
y_xgb_local = y_xgb_local.loc[X_xgb_local.index] # Reajustar y_xgb_local después de eliminar de X_xgb_local

# Tomamos la misma muestra que Random Forest y Gradient Boosting
X_xgb_sample, _, y_xgb_sample, _ = train_test_split(
    X_xgb_local, y_xgb_local, test_size=0.3, random_state=42
)
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb_sample, y_xgb_sample, test_size=0.2, random_state=42
)

# Definimos el modelo XGBoost
xgb_model = XGBRegressor(
    n_estimators=50,   # número de árboles
    max_depth=3,       # profundidad de cada árbol
    learning_rate=0.1, # cada árbol corrige solo parcialmente el error del anterior
    random_state=42,
    n_jobs=-1
)

# Evaluamos el modelo usando la misma función que tus otros modelos
xgb_metrics = evaluate_model(xgb_model, X_train_xgb, y_train_xgb, X_test_xgb, y_test_xgb)

print("XGBoost metrics (sampled data):")
print(xgb_metrics)


XGBoost metrics (sampled data):
{'train_rmse': np.float64(67870.61263048567), 'test_rmse': np.float64(88359.20027945741), 'mae': 23603.603970659864, 'r2': 0.10223430041972348}


## Comparación de modelos

Se comparan los distintos modelos entrenados utilizando métricas de regresión (RMSE, MAE y R²)  
con el objetivo de seleccionar el modelo con mejor rendimiento y menor sobreajuste.


In [18]:
# Comparación de métricas entre modelos
results = pd.DataFrame([
     ["Linear Regression", lr_metrics["test_rmse"], lr_metrics["mae"], lr_metrics["r2"]],
     ["Random Forest", rf_metrics["test_rmse"], rf_metrics["mae"], rf_metrics["r2"]],
     ["Gradient Boosting", gb_metrics["test_rmse"], gb_metrics["mae"], gb_metrics["r2"]],
     ["XGBoost", xgb_metrics["test_rmse"], xgb_metrics["mae"], xgb_metrics["r2"]]
 ], columns=["Model", "RMSE", "MAE", "R2"])


In [19]:
print(results)

               Model          RMSE           MAE        R2
0  Linear Regression  91968.629376  27331.809939  0.027390
1      Random Forest  88644.266750  23539.442706  0.096432
2  Gradient Boosting  89137.414494  23630.931699  0.086351
3            XGBoost  88359.200279  23603.603971  0.102234


#Conclusión:
Linear Regression: Su rendimiento sigue siendo muy bajo (R² ≈ 0.027), indicando que un modelo lineal simple captura muy poca de la compleja relación entre las variables y el precio. Sirve solo como baseline.

Random Forest: Mejora de manera notable respecto a la regresión lineal (R² ≈ 0.096). Reduce RMSE y MAE de manera significativa, mostrando que los modelos basados en árboles pueden capturar mejor la no linealidad del problema.

Gradient Boosting: Rendimiento ligeramente inferior a Random Forest en R² y RMSE. Sigue siendo una opción válida, pero en esta configuración no supera a RF.

XGBoost: Es el modelo con mejor desempeño global en esta comparación (R² ≈ 0.102, RMSE más bajo que RF y GB). Aunque la mejora frente a Random Forest es moderada, XGBoost muestra una mayor capacidad para capturar patrones complejos y es eficiente en entrenamiento.

Decisión: Para este proyecto, XGBoost sería el modelo recomendado como final, por su mayor R² y menor RMSE, mientras que Random Forest se mantiene como un modelo de referencia rápido y robusto. Linear Regression se mantiene como baseline para comparación, y Gradient Boosting como alternativa interesante.

## Validación cruzada (muestra del dataset)

Se aplica validación cruzada (K-Fold) sobre una muestra del dataset para evaluar la estabilidad del modelo seleccionado y detectar posibles problemas de overfitting o underfitting, sin que el entrenamiento se vuelva computacionalmente costoso.

In [20]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# XGBoost Regressor
xgb = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

# Validación cruzada K-Fold con 5 splits
cv_scores = cross_val_score(
    xgb,
    X_train_rf,
    y_train_rf,
    cv=5,
    scoring="r2",
    n_jobs=-1
)

print("R2 scores for each fold:", cv_scores)
print("Mean R2:", cv_scores.mean())
print("Standard deviation:", cv_scores.std())




R2 scores for each fold: [ 0.06379663 -0.06130926 -0.00030879 -0.28541732 -0.05476367]
Mean R2: -0.06760048092885766
Standard deviation: 0.11784898131338425


#Conclusión de la validación cruzada (XGBoost)

El modelo XGBoost Regressor entrenado sobre la muestra muestra un R² promedio negativo (~ -0.068), lo que indica que con estos hiperparámetros y el conjunto de entrenamiento seleccionado, el modelo no logra capturar la variabilidad de los precios y en algunos folds predice peor que un modelo que simplemente predice la media.

Los valores de R² por fold oscilan entre -0.285 y 0.064, mostrando gran variabilidad entre los subsets. Algunos folds incluso presentan R² negativos, lo que evidencia que el modelo no generaliza bien con esta configuración sobre esta muestra.

La desviación estándar (~0.118) es relativamente alta, lo que sugiere que la estabilidad del modelo entre los distintos folds es limitada y que su desempeño es muy sensible a la división de los datos.

En conjunto, estos resultados indican que, con estos hiperparámetros y sobre esta muestra pequeña, XGBoost todavía no logra capturar correctamente la complejidad del dataset. Sería necesario:

Ajustar los hiperparámetros (por ejemplo, aumentar n_estimators, variar max_depth o learning_rate)

Usar una muestra más representativa del dataset completo

Considerar técnicas de regularización y/o features adicionales

Con esto, podemos concluir que el modelo actual de XGBoost no generaliza bien en esta validación cruzada, aunque con tuning adecuado y más datos podría superar a Random Forest y Gradient Boosting.

## Optimización de hiperparámetros

Se realiza una búsqueda de hiperparámetros para mejorar el rendimiento del modelo  
y reducir posibles problemas de sobreajuste.


In [21]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

# Sample + split (igual que antes)
X_gs_sample, _, y_gs_sample, _ = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42
)

X_train_gs, X_test_gs, y_train_gs, y_test_gs = train_test_split(
    X_gs_sample, y_gs_sample, test_size=0.2, random_state=42
)

# Modelo base XGBoost
xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

# Hiperparámetros (acotados para rapidez)
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6],
    "learning_rate": [0.05, 0.1]
}

# GridSearchCV
grid_search = GridSearchCV(
    xgb,
    param_grid,
    cv=3,
    scoring="r2",
    n_jobs=-1
)

grid_search.fit(X_train_gs, y_train_gs)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor R2 (media CV):", grid_search.best_score_)




Mejores hiperparámetros: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
Mejor R2 (media CV): 0.06476429730291126


#Conclusión de la optimización de hiperparámetros (XGBoost)

La búsqueda de hiperparámetros sobre la muestra del dataset ha mostrado que la combinación óptima para el XGBoost Regressor es: learning_rate=0.05, max_depth=4 y n_estimators=100. Con estos parámetros, el modelo presenta un R² medio de aproximadamente 0.065 en validación cruzada, lo que indica que, aunque hemos ajustado los parámetros para mejorar el aprendizaje, el modelo aún explica solo una pequeña parte de la varianza de la variable objetivo en la muestra utilizada. Esto confirma que el problema de predicción del precio es complejo y que la relación entre las variables es relativamente difícil de capturar incluso con un modelo de XGBoost sobre esta muestra.

## Guardado del modelo final

El modelo seleccionado se guarda para su posterior uso en la aplicación interactiva  
desarrollada con Streamlit.


In [22]:
import joblib
from xgboost import XGBRegressor

# Entrenamos el modelo final con los mejores hiperparámetros encontrados
best_xgb = XGBRegressor(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.05,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

best_xgb.fit(X_train_rf, y_train_rf)

# Guardamos el modelo
joblib.dump(best_xgb, "best_xgb_model_sample.pkl")

print("Modelo guardado como 'best_xgb_model_sample.pkl'")




Modelo guardado como 'best_xgb_model_sample.pkl'


El modelo XGBoost Regressor con los mejores hiperparámetros (n_estimators=100, max_depth=4, learning_rate=0.05) ha sido entrenado sobre la muestra y guardado para su uso en la aplicación interactiva. Esto permite cargar el modelo directamente sin necesidad de reentrenarlo cada vez, facilitando la predicción de precios de forma rápida y eficiente.